In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import time

## Read data

In [ ]:
from src.features import load_raw, imply_columns

In [ ]:
df = load_raw()

from src.features import append_features

df['train'], df['submit'] = map(append_features, [df['train'], df['submit']])

cols = imply_columns(df)

In [ ]:
df['train'].shape, df['train'].columns

In [ ]:
df['train'].head()

In [ ]:
df['submit'].shape, df['submit'].columns

In [ ]:
df['submit'].head()

In [ ]:
cols

## train/validation split

In [ ]:
from sklearn.model_selection import train_test_split
test_size=0.3 # 0.3 0. FIXME
x_train, x_test, y_train, y_test = train_test_split(
    df['train'][cols['features']], 
    df['train'][cols['target'][0]], 
    test_size=test_size,
    shuffle=True,
    stratify=df['train'][cols['target'][0]]
)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
sum(y_train), y_train.shape[0], sum(y_test), y_test.shape[0]

In [ ]:
# drop outliers and add columns identifying them
def preprocess(x_in):
    is_outlier = np.tile(np.percentile(x_in, q=95, axis=0), (x_in.shape[0],1))
    x_train2 = x_in.values.copy()
    x_train2 = np.minimum(x_train2, is_outlier)

    #is_outlier = x_train.values > is_outlier
    #x_train2 = np.hstack((x_train2, (is_outlier).astype('uint8')))
    # x_train2[:5]
    return x_train2

# map(...) copied from keras.examples.mnist_net2net
x_train2, x_test2 = map(preprocess, [x_train, x_test])

In [ ]:
x_train2[:5]

In [ ]:
# preprocess for deep network

from sklearn.preprocessing import MinMaxScaler
preprocess = lambda x_in: MinMaxScaler(feature_range=(0, 1), copy=True).fit_transform(x_in)
# map(...) copied from keras.examples.mnist_net2net
x_train2, x_test2 = map(preprocess, [x_train2, x_test2])

In [ ]:
for i in range(len(cols['features'])):
    plt.plot(x_train[cols['features'][i]].values)
    plt.title('before %s'%i)
    plt.show()
    plt.plot(x_train2[:,i])
    plt.title('after %s'%i)
    plt.show()

In [ ]:
import keras
y_train2, y_test2 = map(keras.utils.to_categorical, [y_train, y_test])
y_train2.shape, y_train2[:10]

## keras fully-connected

In [ ]:
from keras.models import Sequential
model = Sequential()

# utility first layer for avoiding need to move input_shape around
from keras.layers import Lambda
model.add(Lambda(lambda x: x, input_shape=(x_train.shape[1],)))

#from keras.layers import BatchNormalization
#model.add(BatchNormalization())

from keras.layers import Dense
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units= 64, activation='relu'))
model.add(Dense(units=  2, activation='softmax'))

model.compile(
              #loss='binary_crossentropy',
              loss='categorical_crossentropy',
              optimizer='sgd',
              # optimizer='adam',
              metrics=['accuracy'])

model.summary()

In [ ]:
# raw data
print(time.ctime(), 'start fit')
history = model.fit(x_train2, y_train2, epochs=1000, batch_size=1024, validation_split=0.1, verbose=0)
print(time.ctime(), 'end')

model.evaluate(x_train2, y_train2)
# [0.47, 0.77]

In [ ]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()

In [ ]:
model.evaluate(x_test2, y_test2)
# [0.48, 0.78] # no added value on top of auto-sklearn